<a href="https://colab.research.google.com/github/AmBha21/sparta-lab/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
from google.colab import drive
drive.mount('/content/drive')

import re
from transformers import BertTokenizer, BertForMaskedLM
import torch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# load BERT large cased model from HuggingFace
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Setting left wing text variable

In [3]:
left_wing_path = '/content/drive/My Drive/sparta lab/transcripts/left_videos.txt'

with open(left_wing_path, "r") as fp:
  text = fp.read().split('\n')

In [5]:
text[:10]

['he',
 'checked off his equipment',
 'hello my name is Abigail',
 'over the last few years I have emailed',
 "Britain's National healthcare provider",
 "133 times trying to get a doctor's",
 'appointment',
 'this video is about what that was like',
 'why big institutions fall apart and why',
 "they're so difficult to change"]

In [ ]:
inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [28]:
left_wing_path = '/content/drive/My Drive/sparta lab/transcripts/left_videos.txt'

with open(left_wing_path, 'r', encoding='utf-8') as file:
    left_wing_text = file.read()
    # print(left_wing_text[:200])
portion_length = len(left_wing_text) // 400

# Extract the 1/20th portion of the text
smaller_text_portion = left_wing_text[:portion_length]
# print(smaller_text_portion)

Making segments for BERT

In [29]:
def segment_and_tokenize(text):
    # Split the text into segments
    sentences = re.split(r'(?<=[.!?])\s+', text)
    segments = []
    current_segment = ""

    for sentence in sentences:
        if len(tokenizer(current_segment + sentence)['input_ids']) > 512:
            segments.append(current_segment)
            current_segment = sentence
        else:
            current_segment += sentence

    segments.append(current_segment)  # Add the last segment
    # print(segments)

    # Tokenize segments
    tokenized_segments = [tokenizer(segment, truncation=True, padding='max_length', max_length=512) for segment in segments]

    return tokenized_segments

# Example usage
tokenized_segments = segment_and_tokenize(smaller_text_portion)


In [36]:
class LeftWingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings_list):
        # Convert list of dicts to dict of lists
        self.encodings = {key: [dic[key] for dic in encodings_list] for key in encodings_list[0]}

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

dataset = LeftWingDataset(tokenized_segments)


In [32]:
# Load the BERT model
model = BertModel.from_pretrained('bert-large-uncased')


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [41]:
def extract_features(dataset):
    model.eval()  # Put the model in evaluation mode
    features = []

    with torch.no_grad():
        for item in dataset:
            # Ensure input has batch dimension
            input_ids = item['input_ids'].unsqueeze(0)
            attention_mask = item['attention_mask'].unsqueeze(0)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            features.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())

    return features

features = extract_features(dataset)
